In [2]:
import os 

In [3]:
%pwd

'c:\\Users\\DELL\\projects\\end-to-end-ML-with-MLflow\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\DELL\\projects\\end-to-end-ML-with-MLflow'

In [6]:
import pandas as pd 

In [7]:
df = pd.read_csv("artifacts\data_ingestion\winequality-red.csv")

In [8]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [9]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [10]:
df["quality"].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         1599 non-null   float64
 1   volatile_acidity      1599 non-null   float64
 2   citric_acid           1599 non-null   float64
 3   residual_sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free_sulfur_dioxide   1599 non-null   float64
 6   total_sulfur_dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [13]:
df.isnull().sum()

fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [14]:
df.shape

(1599, 12)

In [15]:
from dataclasses import  dataclass 
from pathlib import Path
@dataclass 
class DataValidationConfig: 
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict  


In [16]:
from mlproject.constants import * 
from mlproject.utils.common import read_yaml, create_directories

class ConfigurationManager():
    def __init__(self, config_filepath = CONFIG_FILE_PATH, config_params= PARAMS_FILE_PATH, config_schema=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.param  =  read_yaml(config_params)
        self.schema = read_yaml(config_schema)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig: 
        config = self.config.data_validation
        schema = self.schema.COLUMNS 
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema= schema)
        
        return data_validation_config



In [17]:
import os
from mlproject import logger 
import pandas as pd

class DataValidation: 
    def __init__(self, config:DataValidationConfig):
        self.config = config 

    def validate_all_column(self) -> bool: 
        try: 
            validation_status = None 
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()
            data_types = data.dtypes
            all_schema_data_type = list(self.config.all_schema.values())
            for i,col in enumerate(all_cols): 
                if col not in all_schema or data_types[col] != all_schema_data_type[i]:
                    print(data_types[col])
                    print(all_schema_data_type)
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")
                else: 
                    validation_status = True 
                    with open (self.config.STATUS_FILE, "w") as f:
                        f.write(f"Validation status: {validation_status}")
            return validation_status 
            
        except Exception as e: 
            raise e 

In [18]:
try: 
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_column()
except Exception as e: 
    raise e 

[2025-01-03 00:35:50,721 - common - INFO - yaml file: config\config.yaml loadded successfully]
[2025-01-03 00:35:50,724 - common - INFO - yaml file: params.yaml loadded successfully]
[2025-01-03 00:35:50,728 - common - INFO - yaml file: schema.yaml loadded successfully]
[2025-01-03 00:35:50,729 - common - INFO - created directory at: artifacts]
[2025-01-03 00:35:50,730 - common - INFO - created directory at: artifacts/data_validation]


In [19]:
df.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [20]:
for col in df.columns : 
    print(df.dtypes[col]==config.COLUMNS[col])


AttributeError: 'ConfigurationManager' object has no attribute 'COLUMNS'

In [9]:
from pathlib import Path
config = read_yaml(Path("schema.yaml"))
print(config.COLUMNS["fixed acidity"])

[2024-12-30 01:46:26,343 - common - INFO - yaml file: schema.yaml loadded successfully]
float64
